In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\madha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\madha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
'''
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.initializers import RandomNormal

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from keras.callbacks import ModelCheckpoint
'''

'\nfrom keras.preprocessing.text import Tokenizer\nfrom keras.preprocessing.sequence import pad_sequences\n\nfrom keras.models import Sequential\nfrom keras.models import Model\nfrom keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input\nfrom keras.layers.embeddings import Embedding\n\nfrom keras.models import Model\nfrom keras.layers import Input\nfrom keras.layers import Dense\nfrom keras.layers.recurrent import LSTM\nfrom keras.layers.wrappers import TimeDistributed\n\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense, Activation, Dropout\nfrom tensorflow.keras.initializers import RandomNormal\n\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.metrics import log_loss\nfrom keras.callbacks import ModelCheckpoint\n'

Archive:  /content/drive/MyDrive/All/Participants_Data_DCW.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
df = pd.read_csv('train.csv')

In [3]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [4]:
def clean_text(text):
    text = text.lower()
    
    
    text = re.sub(r'@[A-Za-z0-9_]+','', text)
    
    # Dealing with URL links
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex,'urlplaceholder', text)
    # A lot of url are write as follows: http bit.ly. Apply Regex for these cases
    utl_regex_2 = 'http [a-zA-Z]+\.[a-zA-Z]+'
    text = re.sub(utl_regex_2,'urlplaceholder', text)
    # Other formats: http : //t.co/ihW64e8Z
    utl_regex_3 = 'http \: //[a-zA-Z]\.(co|com|pt|ly)/[A-Za-z0-9_]+'
    text = re.sub(utl_regex_3,'urlplaceholder', text)
    
   
    
    
    # Contractions
    text = re.sub(r"what's", 'what is ', text)
    text = re.sub(r"can't", 'cannot', text)
    text = re.sub(r"\'s",' ', text)
    text = re.sub(r"\'ve", ' have ', text)
    text = re.sub(r"n't", ' not ', text)
    text = re.sub(r"im", 'i am ', text)
    text = re.sub(r"i'm", 'i am ', text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", ' would ', text)
    text = re.sub(r"\'ll", ' will ', text)
                  
    # Operations and special words  
    text = re.sub('#',' ', text)         
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub('foof', 'food', text)
    text = re.sub('msg', 'message', text)
    text = re.sub(' u ', 'you', text)
    
    # Ponctuation Removal
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    text = text.split()
    stop_words = stopwords.words("english")
    text = [tok for tok in text if tok not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(w) for w in text]
    return ' '.join(text)

In [5]:
# Cleaning Text
df['Review'] = df['Review'].map(lambda x: clean_text(x))

In [8]:
!pip install --user autokeras 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached autokeras-1.0.16.post1-py3-none-any.whl (166 kB)
  Using cached keras_tuner-1.0.4-py3-none-any.whl (97 kB)
  Using cached tensorflow-2.5.2-cp38-cp38-win_amd64.whl (422.7 MB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached grpcio-1.34.1-cp38-cp38-win_amd64.whl (2.9 MB)
  Using cached h5py-3.1.0-cp38-cp38-win_amd64.whl (2.7 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)


In [6]:
import autokeras as ak

In [7]:
y = np.array(df[[ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']])
X = df['Review']

In [11]:
'''
vectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,4), max_features=300)
vectorizer.fit(X) # fit has to happen only on train data
X = vectorizer.transform(X)
type(X)
'''

'\nvectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,4), max_features=300)\nvectorizer.fit(X) # fit has to happen only on train data\nX = vectorizer.transform(X)\ntype(X)\n'

In [8]:
 Xfeatures = np.array(X)

In [9]:
# Split Data 
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [ ]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=15
)  
# Feed the text classifier with training data.
clf.fit(X_train, y_train, epochs=1)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 4 Complete [00h 36m 45s]
val_loss: 5.977721214294434

Best val_loss So Far: 5.945252418518066
Total elapsed time: 01h 13m 26s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
text_block_1/bl...|bert              |bert              
classification_...|0                 |0                 
optimizer         |adam              |adam_weight_decay 
learning_rate     |2e-05             |2e-05             
text_block_1/ma...|20000             |20000             
text_block_1/be...|512               |512               

 46/108 [===========>..................] - ETA: 19:34 - loss: 5.3712 - accuracy: 0.0041